In [1]:
pip install gradio ollama groq 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import gradio as gr
import ollama
from ollama import Client
from groq import Groq
import os

messages = []

system_prompt = f"""You are an Expert AI agent for managing space missions. You can only answer questions about anomalies in a dataset. You cannot answer questions about anything else. If you receive a question about anomalies in dataset, respond with 'Anomaly' as a single word. If you receive a question that is not about anomalies in a dataset, please respond with 'Other'. 
You have access to these messages + {messages}
You must respond only with one word, 'Anomaly' or 'Other'. You MUST follow the rules."""

client = Groq(api_key="")

def handle_other_messages():
    messages.append({"role": "assistant", "content": "I am an Expert AI agent for managing space missions. I can only answer questions about anomalies in a dataset. I cannot answer questions about anything else. If you have a question about anomalies in a dataset, please ask me. If you have a question that is not about anomalies in a dataset, I cannot help you. Please ask me a question about anomalies in a dataset."})
    return "I am an Expert AI agent for managing space missions. I can only answer questions about anomalies in a dataset. I cannot answer questions about anything else. If you have a question about anomalies in a dataset, please ask me. If you have a question that is not about anomalies in a dataset, I cannot help you. Please ask me a question about anomalies in a dataset."

def get_explanation():
    anomaly = """
    run_id,chan_id,num_train_values,num_test_values,n_predicted_anoms,normalized_pred_error,anom_scores,false_positives,false_negatives,true_positives,fp_sequences,tp_sequences,num_true_anoms,scores,spacecraft,anomaly_sequences,class
    2018-05-19_15.00.10,P-1,2612,8245,4,0.08504268362782921,"[2.0284713382242288, 2.163556600069587, 2.2936227645004643, 0.9175092092214928]",1,0,3,"[[3190, 3329]]","[(2130, 2349), (3540, 3679), (4520, 4589)]",3,"[2.0284713382242288, 2.163556600069587, 2.2936227645004643, 0.9175092092214928]",SMAP,"[[2149, 2349], [4536, 4844], [3539, 3779]]","[contextual, contextual, contextual]"
    2018-05-19_15.00.10,S-1,2558,7071,1,0.19105796561451277,[8.138666940319895],0,0,1,[],"[(5290, 5656)]",1,[8.138666940319895],SMAP,"[[5300, 5747]]",[point]
    2018-05-19_15.00.10,E-1,2620,8256,3,0.06767098449239176,"[3.370114878281593, 8.024256056978087, 9.328760046828243]",1,0,2,"[[1751, 2001]]","[(5010, 5149), (5570, 5965)]",2,"[3.370114878281593, 8.024256056978087, 9.328760046828243]",SMAP,"[[5000, 5030], [5610, 6086]]","[contextual, contextual]"
    """

    messages.append({"role":"user", "content":anomaly})
    messages.append({"role":"user", "content":"Explain the anomalies given above. Explain all the column names and the values."})

    response = client.chat.completions.create(
        model = "llama3-8b-8192",
        messages = messages
    )

    explained_response = response.choices[0].message.content
    return explained_response


def get_message(classification):
    if classification == "Anomaly":
        return get_explanation()
    else:
        return handle_other_messages()

def get_classification(message):
    response = client.chat.completions.create(messages=messages, model="llama3-8b-8192")
    classification = response.choices[0].message.content
    print(classification)
    return get_message(classification)

def predict(message, history):
    messages.append({"role": "user", "content": system_prompt + message})
    classification = get_classification(message)
    return classification


gr.ChatInterface(predict).launch()

AttributeError: module 'gradio' has no attribute 'ChatInterface'